In [1]:
import numpy as np
#import easyocr
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"   # MKL 충돌 잠시 우회
import easyocr  
import time
import ast
import sys      
from datetime import datetime

In [102]:


s = input("변환할 언어 종류 입력 (예: ['ko', 'en']): ").strip()

if not s:                               # 그냥 Enter → 기본값
    langs = ['en']
else:
    try:
        langs = ast.literal_eval(s)     # 문자열 → 리스트
        if not isinstance(langs, list): # 리스트인지 한 번 더 확인
            raise ValueError
    except (ValueError, SyntaxError):
        print("⚠️  리스트 형태로 입력하세요. 예: ['ko', 'en']")
        sys.exit(1)

print(langs)   

g = input("그래픽 사용여부 (1 : 사용, 0 : 미사용): ")#.astype(float)

if g == '':                               # 그냥 Enter → 기본값
    usegpu = True
else:
    try:
        #langs = ast.literal_eval(s)     # 문자열 → 리스트
        usegpu = (float(g) == 1)
        if not isinstance( usegpu, bool): # 리스트인지 한 번 더 확인
            raise ValueError
    except (ValueError, SyntaxError):
        print("0또는 1을 입력하세요 예: 1")
        sys.exit(1)

print( usegpu)   


# 1️⃣ 이미지가 들어 있는 폴더 경로

img_dir = "real_data/"     # ← 자신의 폴더명으로 수정

# 2️⃣ 디렉터리 내 모든 *.png 경로 수집 (정렬 포함)
image_paths = sorted(
    os.path.join(img_dir, fname)
    for fname in os.listdir(img_dir)
    if fname.lower().endswith(".png")           # 확장자 필터
)

print(f"총 {len(image_paths)}장 불러옴")



reader = easyocr.Reader(langs, gpu= usegpu)

변환할 언어 종류 입력 (예: ['ko', 'en']):  


['en']


그래픽 사용여부 (1 : 사용, 0 : 미사용):  


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


True
총 3장 불러옴


In [118]:

t0 = time.perf_counter()

ocr_results = []
for path in image_paths[1:]:
    #result = reader.readtext(path)
    result  = reader.readtext(
    path,
     #'real_data/crop_realdata.png' ,
  allowlist = '0123456789.',
   decoder   = 'beamsearch',
    beamWidth = 1000,
    mag_ratio = 3,
   contrast_ths    = 0.05,
        canvas_size     = 512,
   adjust_contrast = 0.7,
    link_threshold  = 0.4,
    detail    = 1) 
    
    for _, text, conf in result:
        ocr_results.append({
            "filename": path.split('/')[-1],
            "text": text,
            "confidence": round(conf, 2)
        })

df_ocr = pd.DataFrame(ocr_results)

# ── 2. 시간 측정 종료 ──────────────────────────────────────
elapsed = time.perf_counter() - t0
print(elapsed)



# ── 1) OCR 결과를 DataFrame으로 만들기 ───────────────────
df_ocr = pd.DataFrame(ocr_results)    # 앞에서 만든 ocr_results 사용

# ── 2) 결과 폴더 생성 (이미 있으면 건너뜀) ────────────────
result_dir = "result"
os.makedirs(result_dir, exist_ok=True)

# ── 3) 파일 이름(타임스탬프 포함) 설정 ───────────────────
timestamp   = datetime.now().strftime("%Y%m%d_%H%M%S")  # 예: 20250714_105230
file_name   = f"eaysocr_results_{timestamp}.csv"
file_path   = os.path.join(result_dir, file_name)

# ── 4) 저장 ───────────────────────────────────────────────
df_ocr.to_csv(file_path, index=False, encoding="utf-8-sig")  # Excel에서 한글 깨짐 방지

print(f"✅ OCR 결과가 저장되었습니다 ➜ {file_path}")

C:\Users\USER\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


0.9783154999986436
✅ OCR 결과가 저장되었습니다 ➜ result\eaysocr_results_20250714_132157.csv
